In [1]:
from __future__ import print_function

import numpy as np
import glob


## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        

from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, RunSummary, \
                                         Inputs, \
                                         get_full_id_from_partial_id

from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs, \
                                                Simulation_Status


/home/egentry/anaconda3/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/media/egentry/Windows7_OS/Users/Eric/Dropbox/data/clustered_SNe/analysis/../../clustered_SNe/analysis/database_helpers.py:41: UserWarning: `session' from database_helpers can only write using 1 process at a time
  UserWarning)


In [2]:
data_dir = "../saved_runs/cluster_parameter_study/"

In [3]:
ids = [
"32226257-da2d-47ae-85ef-d99b39621202",
"20b4fccc-e53f-48ca-894b-3ee282bad8ea",
"3cf9d3f4-b657-4437-a59f-324de0b52763",
"db48961d-c5d2-4495-8c47-a1922d0170a9",
"45a6c151-774b-479f-b4fc-83c9945516c7",
"b7ef7e0d-72b5-4592-ad9a-288fa98fd7cc",
"af79be27-ff8e-47c1-bee1-e29a2786d908",
"e219741b-5ff9-488c-8a0b-c64b6f568c00",
"9504c81d-b2f7-4593-8c28-e661c501cb11",
"c436d5db-4feb-41a0-8175-f6dd2287a6b9",
"e3d7135a-0265-4216-b63f-e29ba039f912",
"c7b202a2-dbf2-46fb-a192-4ab258b70b9d"
]

# ids = [get_full_id_from_partial_id(data_dir, partial_id) for partial_id in ids]


In [4]:
# in_par_restart_template = """
# //Time Range//

# T_Start             1e0
# T_End               {0:.2e}


# //Output Frequency//

# Num_Reports         1000
# Num_Checkpoints     {1:d}
# Use_Logtime         true


# //Grid Dimensions//

# Num_R               1024
# R_Min               3.08e16
# R_Max               3.08e20
# Log_Zoning          0        //0=Uniform, 1=Log, 2=Hybrid
# Log_Radius          0.25     //Only for option 2 of Log_Zoning

# Max_Aspect_Short    10.0
# Max_Aspect_Long     10.0     //Aspect Ratio AMR Refinement Criteria


# //Numerical Parameters//

# CFL                 {2:f}
# PLM                 1
# RK2                 1
# H_0                 0.0
# H_1                 0.1
# Riemann_Solver      1       //0=HLL, 1=HLLC
# Mesh_Motion         1       //0=Fixed, 1=By Cell, 2=Riemann Solver
# Density_Floor       1e-60
# Pressure_Floor      1e-40 


# //Cooling Parameters//

# With_Cooling        true
# Cooling_Type        equilibrium


# //Hydro Parameters//

# Adiabatic_Index     1.666666667


# //Initial Conditions//

# ICs                 restart


# //Feedback//

# mass_loss           uniform
# """

# restart_batch_template = """#!/bin/bash
# #
# #$ -M egentry@ucsc.edu
# #$ -m e
# #
# #$ -cwd
# #$ -S /bin/bash
# #

# source $HOME/.bashrc
# time ../bin/SNe ../src/in.par.restart.{0} {0}"""

In [9]:
qsub_system = "SGE"

paths = glob.glob("../src/tmp_in_par/*")
for path in paths:
    os.remove(path)

paths = glob.glob("../scripts/" + qsub_system + "/tmp_restart/*")
for path in paths:
    os.remove(path)

f_delete = open("delete_script", mode="w")
f_delete.write("#!/bin/bash \n")
f_delete.write("mkdir -p _tmp_ \n")

    
f_submit = open("submit_all", mode="w")
f_submit.write("#!/bin/bash \n")

for id in ids:

    print("id: ", id)
    run_summary = RunSummary(data_dir=data_dir, id=id)

    bad_checkpoint_name = run_summary.first_unreasonable_energy()
    if bad_checkpoint_name is None:
        continue
    bad_checkpoint_num = int(bad_checkpoint_name.split("_")[-1].strip(".dat"))
    restart_checkpoints_needed = 100 - (bad_checkpoint_num % 100)
    print("bad_checkpoint_num: ", bad_checkpoint_num)
    print("new checkpoints_needed: ", restart_checkpoints_needed)
    
    bad_checkpoint_index = np.argmax(np.array(run_summary.filenames)==bad_checkpoint_name)
    for filename in run_summary.filenames[:bad_checkpoint_index]:
        f_delete.write("mv       " + os.path.basename(filename) + " _tmp_/ \n")
    f_delete.write("rm -f    " + id + "_checkpoint* " + "\n")
    f_delete.write("mv _tmp_/" + id + "* . \n")
    
    restart_time_needed = run_summary.overview.SNe_times.max() * 3 - run_summary.times[bad_checkpoint_index]
    if bad_checkpoint_num > 99:
        restart_time_needed = 3e15
    print("extra time needed: ", restart_time_needed, "[s]")

    print("")

    CFL = .025

#     with open("../src/tmp_in_par/in.par.restart." + run_summary.id, mode="w") as f:
#         f.write(in_par_restart_template.format(restart_time_needed, restart_checkpoints_needed, CFL))

#     with open("../scripts/SGE/tmp_restart/restart.batch." + run_summary.id, mode="w") as f:
#         f.write(restart_batch_template.format(run_summary.id))
        
    f_submit.write("qsub " +
                       "-v RESTART_ID=" + run_summary.id + ","
                       + "RESTART_N_CHECKPOINTS=" + str(restart_checkpoints_needed) + ","
                       + "RESTART_DELTA_TIME=" + str(restart_time_needed) + ","
                       + "RESTART_CFL=" + str(CFL)
                    + "../scripts/" + qsub_system + "/restart.batch.env_variables \n")
        
f_delete.write("rm -rf _tmp_ \n")
f_delete.close()
f_submit.close()

id:  32226257-da2d-47ae-85ef-d99b39621202
bad_checkpoint_num:  2
new checkpoints_needed:  97
extra time needed:  2.8148237e+15 [s]

id:  20b4fccc-e53f-48ca-894b-3ee282bad8ea
bad_checkpoint_num:  64
new checkpoints_needed:  35
extra time needed:  1.6281142e+15 [s]

id:  3cf9d3f4-b657-4437-a59f-324de0b52763
bad_checkpoint_num:  38
new checkpoints_needed:  61
extra time needed:  2.307064e+15 [s]

id:  db48961d-c5d2-4495-8c47-a1922d0170a9
bad_checkpoint_num:  86
new checkpoints_needed:  13
extra time needed:  5.98064e+14 [s]

id:  45a6c151-774b-479f-b4fc-83c9945516c7
bad_checkpoint_num:  2
new checkpoints_needed:  97
extra time needed:  1.9919052e+15 [s]

id:  b7ef7e0d-72b5-4592-ad9a-288fa98fd7cc
bad_checkpoint_num:  67
new checkpoints_needed:  32
extra time needed:  2.485415e+14 [s]

id:  af79be27-ff8e-47c1-bee1-e29a2786d908
bad_checkpoint_num:  85
new checkpoints_needed:  14
extra time needed:  8.835297e+14 [s]

id:  e219741b-5ff9-488c-8a0b-c64b6f568c00
bad_checkpoint_num:  86
new checkp

In [12]:
ids

['32226257-da2d-47ae-85ef-d99b39621202',
 '20b4fccc-e53f-48ca-894b-3ee282bad8ea',
 '3cf9d3f4-b657-4437-a59f-324de0b52763',
 'db48961d-c5d2-4495-8c47-a1922d0170a9',
 '45a6c151-774b-479f-b4fc-83c9945516c7',
 'b7ef7e0d-72b5-4592-ad9a-288fa98fd7cc',
 'af79be27-ff8e-47c1-bee1-e29a2786d908',
 'e219741b-5ff9-488c-8a0b-c64b6f568c00',
 '9504c81d-b2f7-4593-8c28-e661c501cb11',
 'c436d5db-4feb-41a0-8175-f6dd2287a6b9',
 'e3d7135a-0265-4216-b63f-e29ba039f912',
 'c7b202a2-dbf2-46fb-a192-4ab258b70b9d']

In [6]:
for id in ids:
#     full_id = get_full_id_from_partial_id(data_dir, id)
    session.query(Simulation_Status).filter(Simulation_Status.id == id).update({"status":"Running"})
    sim_status = session.query(Simulation_Status).get(id)
    print(sim_status.status)
    print(id)

Running
32226257-da2d-47ae-85ef-d99b39621202
Running
20b4fccc-e53f-48ca-894b-3ee282bad8ea
Running
3cf9d3f4-b657-4437-a59f-324de0b52763
Running
db48961d-c5d2-4495-8c47-a1922d0170a9
Running
45a6c151-774b-479f-b4fc-83c9945516c7
Running
b7ef7e0d-72b5-4592-ad9a-288fa98fd7cc
Running
af79be27-ff8e-47c1-bee1-e29a2786d908
Running
e219741b-5ff9-488c-8a0b-c64b6f568c00
Running
9504c81d-b2f7-4593-8c28-e661c501cb11
Running
c436d5db-4feb-41a0-8175-f6dd2287a6b9
Running
e3d7135a-0265-4216-b63f-e29ba039f912
Running
c7b202a2-dbf2-46fb-a192-4ab258b70b9d


In [8]:
session.commit()